In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
import xgboost as xgb
import pymorphy2
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
tnz = RegexpTokenizer('\w+|[^\w\s]+')
morph = pymorphy2.MorphAnalyzer()
import nltk
nltk.download('stopwords')
import string
def tokenize_me(file_text):
    tokens = nltk.word_tokenize(file_text)
    tokens = [morph.parse(i.lower())[0].normal_form for i in tokens if ( i not in string.punctuation )]
 
    stop_words = stopwords.words('russian')+stopwords.words('english')
    stop_words.extend(map(lambda x: x, 
                          ['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на']))
    tokens = [i for i in tokens if ( i not in stop_words )]     
    return ' '.join(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vkrin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df = pd.read_csv('data.csv', encoding = 'UTF-8', sep = ';')

In [4]:
df.head(10)

,ID,Question
0,1,Как зовут лодочника на реке Стикс в древнегреч...
1,2,Как в химии обозначается свинец?
2,3,Какой химический элемент преобладает в составе...
3,4,Кто из перечисленных был пажом во времена Екат...
4,5,Когда началась 2 мировая война?
5,6,Какая эра геологической истории Земли относитс...
6,7,Кто из животных может мурлыкать?
7,8,Что из перечисленного НЕ входит в национальную...
8,9,Какой из этих мультфильмов повествует о реальн...
9,10,Кто написал повесть «Муму»


In [7]:
quetions = df['Question'].values

In [6]:
from stop_words import get_stop_words
sw= get_stop_words('ru')

In [11]:
quetions_normal=[]
for i in quetions:
    quetions_normal.append(tokenize_me(i))

In [26]:
tfidf = TfidfVectorizer(ngram_range=(1,5))
x = tfidf.fit_transform(quetions_normal)

In [27]:
x.shape

(41087, 343508)

In [15]:
df_train = pd.read_csv('train.csv', encoding = 'UTF-8', sep = ';')

In [16]:
df_train.head()

,ID,Answer
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0


In [28]:
x_train = x[:30000]
x_test=x[30000:]

In [18]:
x_train.shape

(30000, 483100)

In [19]:
y_train = df_train["Answer"]

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x1, x2, y1, y2 = train_test_split(x_train,y_train, test_size = 0.2)

In [29]:
model = xgb.XGBClassifier(max_depth=5, n_jobs=4, n_estimators=500)

In [30]:
model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=4,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [26]:
from sklearn.metrics import roc_auc_score

In [31]:
y_pred2 = model.predict_proba(x_test)

In [32]:
y_pred2

array([[0.93262744, 0.06737256],
       [0.96674573, 0.03325425],
       [0.9241952 , 0.07580482],
       ...,
       [0.9304354 , 0.06956461],
       [0.85126925, 0.14873078],
       [0.8912035 , 0.10879648]], dtype=float32)

In [53]:
roc_auc_score(y_pred2[:,1], y_pred)

ValueError: continuous format is not supported

In [25]:
out=open('sub.csv','w', encoding='utf-8')
for i in range (len(y_pred2)):
    out.writelines(str(30000+i+1)+','+str(y_pred2[i][1])+'\n')
out.close()

array([0.08086899, 0.08086899, 0.13753864, ..., 0.08086899, 0.08086899,
       0.08086899], dtype=float32)